In [ ]:
# Imports
from snowflake.snowpark.session import Session
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder
from snowflake.ml.modeling.metrics import r2_score, accuracy_score, precision_score, roc_auc_score, f1_score, recall_score
from snowflake.snowpark.functions import col, is_null, regexp_replace, when, lit
from snowflake.snowpark.types import StringType
from snowflake.snowpark.exceptions import SnowparkSQLException
import importlib, sys, json, os, logging


In [ ]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}